In [1]:
import numpy as np
import pandas as pd

In [10]:
all_airlines = pd.read_csv("dataset/archive/Airlines.csv")
print(all_airlines.head())
print(all_airlines.describe())

  Code                    Description
0  02Q                  Titan Airways
1  04Q             Tradewind Aviation
2  05Q            Comlux Aviation, AG
3  06Q  Master Top Linhas Aereas Ltd.
4  07Q            Flair Airlines Ltd.
        Code    Description
count   1570           1571
unique  1570           1571
top      02Q  Titan Airways
freq       1              1


In [12]:
flight_data = pd.read_csv("dataset/archive/Combined_Flights_2022.csv")
print(all_airlines.head())
print(all_airlines.describe())

  Code                    Description
0  02Q                  Titan Airways
1  04Q             Tradewind Aviation
2  05Q            Comlux Aviation, AG
3  06Q  Master Top Linhas Aereas Ltd.
4  07Q            Flair Airlines Ltd.
        Code    Description
count   1570           1571
unique  1570           1571
top      02Q  Titan Airways
freq       1              1
